# Formatting Tables in Pandas


In [145]:
# Import pandas
import pandas as pd
# Import numpy
import numpy as np



## The data
Let's create some simulated data for two widgets, A and B. We'll create a dataframe for each widget, then concatenate them together. We'll also sort the dataframe by month and reset the index.

In [146]:
# simulated data for widget A
df_a = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 1_000_000,
            high = 2_500_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 300_000,
            high = 500_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 750_000,
            high = 1_250_000,
            size = 132
        )
    }
)

df_a['Product'] = 'A'

# simulated data for widget B
df_b = pd.DataFrame(
    {
        'Month':pd.date_range(
            start = '01-01-2012',
            end = '31-12-2022',
            freq = 'MS'
        ),
        'Quotes':np.random.randint(
            low = 100_000,
            high = 800_000,
            size = 132
        ),
        'Numbers':np.random.randint(
            low = 10_000,
            high = 95_000,
            size = 132
        ),
        'Amounts':np.random.randint(
            low = 450_000,
            high = 750_000,
            size = 132
        )
    }
)

df_b['Product'] = 'B'

# put it together & sort
df = pd.concat([df_a,df_b],axis = 0)
df.sort_values(by = 'Month',inplace = True)
df.reset_index(drop = True,inplace = True)

Let’s calculate a few “interesting” statistics — average sale amounts and product conversion:

In [147]:
# average sale
df['Average sale'] = df['Amounts'] / df['Numbers'].replace({0: np.nan})

# conversion
df['Product conversion'] = df['Numbers'] / df['Quotes'].replace({0: np.nan})

In [148]:
df.head(3)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01-01,1620435,361730,985417,A,2.724178,0.223230
1,2012-01-01,428907,92636,485731,B,5.243437,0.215982
2,2012-02-01,2437497,422517,780098,A,1.846312,0.173341


## Date Formatting
There’s arguably nothing __wrong__ with the formatting, but it could be better. For instance, since all the monthly data is reflected as at the first of each month, there’s probably little sense in keeping the day element of each Month entry as it tells the reader very little.

In [149]:
# format the date as YYYY-MM
styler = df.iloc[:3].style.format({'Month':'{:%Y-%m}'})
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,2012-01,1620435,361730,985417,A,2.724178,0.223230
1,2012-01,428907,92636,485731,B,5.243437,0.215982
2,2012-02,2437497,422517,780098,A,1.846312,0.173341


Now, we can improve readability even further by using the name of each month rather than the month number, and we can do this __*without having to alter the underlying data*__.

In [150]:
styler = df.iloc[:3].style.format({'Month':'{:%B %Y}'})
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,January 2012,1620435,361730,985417,A,2.724178,0.223230
1,January 2012,428907,92636,485731,B,5.243437,0.215982
2,February 2012,2437497,422517,780098,A,1.846312,0.173341


Maybe a little too wordy now — let’s use abbreviations instead (e.g. “Jan” instead of “January”) and we’ll also add a comma before the year.

In [151]:
styler = df.iloc[:3].style.format({'Month':'{:%b, %Y}'})
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012",1620435,361730,985417,A,2.724178,0.223230
1,"Jan, 2012",428907,92636,485731,B,5.243437,0.215982
2,"Feb, 2012",2437497,422517,780098,A,1.846312,0.173341


## Formatting numbers with a thousand separator
A fairly straightforward formatting experience here as we separate thousands of Quotes and Numbers using commas.

What is important to note however, is that if we also want to retain the formatting we applied to the Month column (we do), then we need to extend the formatting dictionary. 


In [163]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}'
    }
)
if styler:
    display(styler)

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012","1,620,435","361,730",985417,A,2.724178,0.223230
1,"Jan, 2012","428,907","92,636",485731,B,5.243437,0.215982
2,"Feb, 2012","2,437,497","422,517",780098,A,1.846312,0.173341


## Formatting currencies
The Widget Company just so happens to produce and sell its widgets in a country that uses a currency denoted by £ (I hope somewhere warmer and sunnier than the country where I earn my £).

Let’s reflect that in the table, reminding ourselves that:

- At an overall level, using decimal points is probably a little much
- At a lower level — say for instance, the average sale value — using decimals can be useful.
So we add currency formatting for Amounts and Average sale to our formatting dictionary:

In [162]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}'
    }
)
if styler:
    display(styler)
    
breakpoint()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012","1,620,435","361,730","£985,417",A,£2.72,0.223230
1,"Jan, 2012","428,907","92,636","£485,731",B,£5.24,0.215982
2,"Feb, 2012","2,437,497","422,517","£780,098",A,£1.85,0.173341


## Formatting percentages
Another fairly straightforward formatting step, it’s much easier to view ratios when they are expressed as percentages rather than floating point numbers.

We’ll add Product conversion to our formatting dictionary:

In [160]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}',
        'Product conversion':'{:.2%}'
    }
)
if styler:
    display(styler)
    
breakpoint()

,Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
0,"Jan, 2012","1,620,435","361,730","£985,417",A,£2.72,22.32%
1,"Jan, 2012","428,907","92,636","£485,731",B,£5.24,21.60%
2,"Feb, 2012","2,437,497","422,517","£780,098",A,£1.85,17.33%


## Hiding Indexes
The index is a useful thing to have, but it’s not always necessary to display it. In this case, we can hide it by setting index = False:


In [155]:
styler = df.iloc[:3].style.format(
    {
        'Month':'{:%b, %Y}',
        'Quotes':'{:,.0f}',
        'Numbers':'{:,.0f}',
        'Amounts':'£{:,.0f}',
        'Average sale':'£{:,.2f}',
        'Product conversion':'{:.2%}'
    }
).hide()  # hide the index

if styler:
    display(styler)
    
breakpoint()


Month,Quotes,Numbers,Amounts,Product,Average sale,Product conversion
"Jan, 2012","1,620,435","361,730","£985,417",A,£2.72,22.32%
"Jan, 2012","428,907","92,636","£485,731",B,£5.24,21.60%
"Feb, 2012","2,437,497","422,517","£780,098",A,£1.85,17.33%


## Conditional Formatting
Conditional formatting is a great way to highlight certain values in a table. For instance, we might want to highlight the highest and lowest values in each column.
Let's start with highlighting rows if an element of the row meets a given condition — in this case, highlight all rows containing information relating to product A.

We do this in two steps:

1. Define the function highlight_product which returns a string if the given condition is met (that is, if the row relates to the specified product). The string contains a format command that we will pass through to the Styler.
2. The resulting format command is fed through using the apply command.



In [165]:
def highlight_product(s, product,colour = 'yellow'):
    r = pd.Series(data = False,index = s.index)
    r['Product'] = s.loc['Product'] == product
    
    return [f'background-color: {colour}' if v==True else '' for v in r['Product']]
    
styler = df.iloc[:3].style\
    .apply(  # apply the conditional formatting
        highlight_product,
        product = 'A',
        axis = 1
    ).format(  # format the table
        {
            'Month':'{:%b, %Y}',
            'Quotes':'{:,.0f}',
            'Numbers':'{:,.0f}',
            'Amounts':'£{:,.0f}',
            'Average sale':'£{:,.2f}',
            'Product conversion':'{:.2%}'
        }
    ).hide()  # hide the index  

if styler:
    display(styler)
    
breakpoint()   

TypeError: 'numpy.bool_' object is not iterable